In [77]:
%pip install -qU pypdf 
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



In [78]:
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv
file_path = "./solving_sudoku.pdf"
loader = PyPDFLoader(file_path)

load_dotenv()
docs = loader.load()

In [79]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = os.getenv("API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [40]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

In [85]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for solving sudoku."
    "You will be provided with the current state of the puzzle as well as the solution of the puzzle." 
    "If there are any mistakes in the current puzzle, point them out and end. "
    "Otherwise, choose a random cell that you think can be deduced from the current state of the puzzle and decide which number belongs there."
    #"Verify that the number you deduced is correct, using the solution. "
    "Explain the logic that led you to that number, without referencing the solution."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [46]:
from sudoku import Sudoku

In [80]:
new_sudoku = Sudoku(3).difficulty(0.4)
new_sudoku.board[8][4] = 7;
new_sudoku.show()

Puzzle has exactly one solution
+-------+-------+-------+
| 5 2 4 |   1   | 9   6 |
| 7 9   | 5     | 4 8 3 |
| 3   8 |   9 7 |       |
+-------+-------+-------+
|   4 5 | 7   1 | 3     |
| 6 7   | 2 4 9 | 5 1   |
|   8 9 | 6 3   |     7 |
+-------+-------+-------+
|   3 6 |   2   |   5   |
| 9 5   |   7 6 | 8 3   |
| 4 1 7 | 3 7 8 | 6     |
+-------+-------+-------+



In [86]:
results = rag_chain.invoke({"input": str(new_sudoku.board)+str(new_sudoku.solve().board)})
print(results['answer'])

There is a mistake in the current state of the puzzle. Specifically, the last row contains the numbers [4, 1, 7, 3, 7, 8, 6, None, None]. Notice that the number 7 appears twice in this row, which violates the Sudoku rule that each number from 1 to 9 must appear exactly once in each row. This needs to be corrected before proceeding with solving the puzzle.


In [82]:
solution = new_sudoku.solve()
solution.show()

Puzzle has no solution
+-------+-------+-------+
|       |       |       |
|       |       |       |
|       |       |       |
+-------+-------+-------+
|       |       |       |
|       |       |       |
|       |       |       |
+-------+-------+-------+
|       |       |       |
|       |       |       |
|       |       |       |
+-------+-------+-------+

